# Modifying MCMC Initial Positions

by Henry Ngo (2019)

When you set up the MCMC Sampler, the initial position of your walkers are randomly determined. Specifically, they are uniformly distributed in your Prior phase space. This tutorial will show you how to change this default behaviour so that the walkers can begin at locations you specify. For instance, if you have an initial guess for the best fitting orbit and want to use MCMC to explore posterior space around this peak, you may want to start your walkers at positions centered around this peak and distributed according to an N-dimensional Gaussian distribution. 

Note: This tutorial is meant to be read after reading the [MCMC Introduction tutorial](https://orbitize.readthedocs.io/en/latest/tutorials/MCMC_tutorial.html). If you are wondering what walkers are, you should start there!

The `Driver` class is the main way you might interact with `orbitize!` as it automatically reads your input, creates all the `orbitize!` objects needed to do your calculation, and defaults to some commonly used parameters or settings. However, sometimes you want to work directly with the underlying API to do more advanced tasks such as changing the MCMC walkers' initial positions, or [modifying the priors](https://orbitize.readthedocs.io/en/latest/tutorials/Modifying_Priors.html).

This tutorial walks you through how to do that. 

**Goals of this tutorial**:
- Learn to modify the MCMC `Sampler` object
- Learn about the structure of the `orbitize` code base

## Import modules

In [2]:
import numpy as np

import orbitize
from orbitize import driver
import multiprocessing as mp

## Create Driver object

First, let's begin as usual and create our `Driver` object, as in the MCMC Introduction tutorial.

In [4]:
filename = "{}/GJ504.csv".format(orbitize.DATADIR)

# system parameters
num_secondary_bodies = 1
system_mass = 1.75 # [Msol]
plx = 51.44 # [mas]
mass_err = 0.05 # [Msol]
plx_err = 0.12 # [mas]

# MCMC parameters
num_temps = 5
num_walkers = 30
num_threads = mp.cpu_count() # or a different number if you prefer


my_driver = driver.Driver(
    filename, 'MCMC', num_secondary_bodies, system_mass, plx, mass_err=mass_err, plx_err=plx_err,
    mcmc_kwargs={'num_temps': num_temps, 'num_walkers': num_walkers, 'num_threads': num_threads}
)

## Access the `Sampler` object to view the walker positions

As mentioned in the introduction, the `Driver` class creates the objects needed for the orbit fit. At the time of this writing, it creates a `Sampler` object which you can access with the `.sampler` attribute and a `System` object which you can access with the `.system` attribute.

The `Sampler` object contains all of the information used by the orbit sampling algorithm (OFTI or MCMC) to fit the orbit and determine the posteriors. The `System` object contains information about the astrophysical system itself (stellar and companion parameters, the input data, etc.). 

To see all of the attributes of the driver object, you can use `dir()`.

In [18]:
dir(my_driver)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'sampler',
 'system']

This returns many other functions too, but you see `sampler` and `system` at the bottom. Don't forget that in Jupyter notebooks, you can use `my_driver?` to get the docstring for its class (i.e. the `Driver` class) and `my_driver??` to get the full source code of that class. You can also get this information in the API documentation.

Now, let's list the attributes of the `my_driver.sampler` attribute.

In [19]:
dir(my_driver.sampler)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_fill_in_fixed_params',
 '_logl',
 'curr_pos',
 'fixed_params',
 'lnlike',
 'num_params',
 'num_temps',
 'num_threads',
 'num_walkers',
 'priors',
 'results',
 'run_sampler',
 'system',
 'use_pt']

Again, you can use the `?` and `??` features as well as the API documentation to find out more. Here we see an attribute `curr_pos` which contains the current position of all the walkers for the MCMC sampler. These positions were generated upon initialization of the `Sampler` object, which happened as part of the initialization of the `Driver` object.

### Examine `my_driver.sampler.curr_pos`

`curr_pos` is an array and has shape (`n_temps`, `n_walkers`, `n_params`) for the parallel-tempered MCMC sampler and shape (`n_walkers`, `n_params`) for the affine-invariant ensemble sampler. 

In [28]:
my_driver.sampler.curr_pos.shape # Here we are using the parallel-tempered MCMC sampler

(5, 30, 8)

Basically, this is the same shape as the output of the Sampler. Let's look at the start position of the first five walkers at the lowest temperature, to get a better sense of what the strucutre is like.

In [29]:
print(my_driver.sampler.curr_pos[0,0:5,:])

[[ 0.24504151  0.63705443  1.77924184  3.18908565  3.22058099  0.33553179
  51.39646203  1.71189669]
 [ 5.91511047  0.22330853  1.60508749  0.63809505  1.5515721   0.8469325
  51.56434961  1.82177098]
 [ 0.39371766  0.69909325  1.34413767  4.64406445  5.67560083  0.31355571
  51.44936553  1.71900705]
 [14.77243131  0.67288329  2.91407727  2.51512375  4.7190362   0.51339391
  51.52403468  1.73689092]
 [ 4.56868726  0.22396506  1.44092969  2.0759432   1.84652498  0.31772185
  51.36397433  1.72585335]]


## Replace `curr_pos` with your own initial positions for walkers

When the sampler is run with the `sampler.run_sampler()` method, it will start the walkers at the `curr_pos` values, run the MCMC forward for the given number of steps, and then update `curr_pos` to reflect where the walkers ended up. The next time `run_sampler()` is called, it does the same thing again.

Here, we have just created the sampler but have not run it yet. So, if we update `curr_pos` with our own custom start locations, when we run the sampler, it will begin at our custom start locations instead.

### Generate your own initial positions

There are many ways to create your own walker start distribution and what you want to do will depend on your science question and prior knowledge. Here, let's assume we know a possible best fit value and our uncertainty in that fit. So, we will try to create a distribution of walkers that are centered on the best fit value and distributed normallly with the 1-sigma in each dimension equal to the uncertainty on that best fit value.

First, let's define the best fit value and the spread. As a reminder, the order of the parameters in the array is: semimajor axis, eccentricity, inclination, argument of periastron, position angle of nodes, epoch of periastron passage, parallax and total mass. If there are more than one body, the first six elements are repeated for the second and subsequent body right after the first body so that parallax and total mass is always the last two elements. You can check the indices with this dict in the `system` object.

In [39]:
print(my_driver.system.param_idx)

{'sma1': 0, 'ecc1': 1, 'inc1': 2, 'aop1': 3, 'pan1': 4, 'epp1': 5, 'plx': 6, 'mtot': 7}


In [43]:
# Set centre and spread of the walker distribution
# Values from Table 1 in Blunt et al. 2017, AJ, 153, 229
sma_cen = 44.48
sma_sig = 15.0
ecc_cen = 0.0151
ecc_sig = 0.175
inc_cen = 131.7
inc_sig = 16.0
aop_cen = 91.7
aop_sig = 60.0
pan_cen = 133.7
pan_sig = 50.0
epp_cen = 2228.11
epp_sig = 121.0
# Note : parallax and system mass already defined above (plx, plx_err, system_mass, mass_err)
walker_centres = np.array([sma_cen,ecc_cen,inc_cen,aop_cen,pan_cen,epp_cen,plx,system_mass])
walker_1sigmas = np.array([sma_sig,ecc_sig,inc_sig,aop_sig,pan_sig,epp_sig,plx_err,mass_err])

We can use `numpy.random.standard_normal` 

In [46]:
curr_pos_shape = my_driver.sampler.curr_pos.shape # Get shape of walker positions
# Draw from multi-variate normal distribution to generate new walker positions
new_pos = np.random.standard_normal(curr_pos_shape)*walker_1sigmas + walker_centres

In [ ]:
# add 180 to Omega